# Creating submission workflows using qcsubmit.

Here our aim is to create reproducible workflows which can process large lists of molecules applying filtering and some other useful operations 
such as state enumeration and fragmentation before formatting the data into a qcarchive dataset that can be 
submitted to the public or local archive instance. 

The entire workflow is build up out of modular components which can each have their own programable settings which can be 
controlled through the API or via workflow yaml/json files. 

In this example we will demonstrate how to set up a basic workflow via the API and how it can be exported to a settings file 
which can then be used to reconstruct the entire workflow by another user.

First lets load in the packages.

In [1]:
from qcsubmit.factories import QCFractalDatasetFactory  # load in the factory to process molecules
from qcsubmit import workflow_components                # load in a list of workflow_components
from openforcefield.utils.utils import get_data_file_path  # a util function to load a mini drug bank file
from openforcefield.topology import Molecule


RDKit WARNING: [16:35:15] Enabling RDKit 2019.09.3 jupyter extensions


Create the basic qcarchive dataset factory, this is useful for large machine learning datasets which are a collection
of single point calculations provided through the energy/gradient/hessian drivers.

In [2]:
factory = QCFractalDatasetFactory()
# we can view and change any of the basic settings in the factory such as the qm settings
factory.theory = 'wB97x-d'
# lets look at the class and the settings 
print(factory)

theory='wB97x-d' basis='DZVP' program='psi4' maxiter=200 driver='energy' scf_properties=['dipole', 'qudrupole', 'wiberg_lowdin_indices'] client='public' priority='normal' tag='openff' workflow={}


In [3]:
# each of the fields are also validated as the class is based on pydantic
# so the following should produce and error
factory.basis = {'test': 1}


ValidationError: 1 validation error for QCFractalDatasetFactory
basis
  str type expected (type=type_error.str)

Note that the basic settings should be suitable in most cases as they are those recommended by the 
openforcefield and are currently used in the fitting of the most recent force fields. Now lets look at
the workflow components. 

In [4]:
# the workflow is a dictionary that contains all of the components that will be executed in order
print(factory.workflow)

{}


In [5]:
# the workflow is also validated so only properly configured workflow components can be added
factory.add_workflow_component(3)

InvalidWorkflowComponentError: Component 3 rejected as it is not a sub class of CustomWorkflowComponent.

As you can see the number is not a proper workflow component and so it has been rejected from the workflow.
Users can make there own workflow components which can be added to the workflow as well but they must be a subclass
of the CustomWorkflowComponent class and have all of the abstract methods implemented and settings. See the creating workflow
components notebook for examples on how to do this.

Now lets set up a workflow that will filter out some unwanted elements, then filter by molecular weight
before generating conformers for each of the molecules.

In [6]:
# set up the element filter
el_filter = workflow_components.ElementFilter()
# lets view the options available for this filter
print(el_filter)

component_name='ElementFilter' component_description='Filter out molecules who contain elements not in the allowed element list' component_fail_message='Molecule contained elements not in the allowed elements list' allowed_elements=['H', 'C', 'N', 'O', 'F', 'P', 'S', 'Cl', 'Br', 'I']


This filter has the ability to filter elements by atomic name or number, we just have to supply a list of 
symbols or numbers to the filter. Here lets only keep molecules with elements of H,C,N and O as we would like
to use AN1 as our QM method.

In [7]:
# set the filter to only keep molecules with these elements
el_filter.allowed_elements = [1, 6, 7, 8]

# now lets add the filter to the workflow
factory.add_workflow_component(el_filter)

Now we will set up the weight filter and conformer generation components and add them to the workflow.

In [8]:
weight_filter = workflow_components.MolecularWeightFilter()
factory.add_workflow_component(weight_filter)
conf_gen = workflow_components.StandardConformerGenerator()
factory.add_workflow_component(conf_gen)

Now lets look at the workflow and make sure all of the components were added in correctly. Then lets save the 
settings and workflow so they can be used again latter.


In [9]:
print(factory.workflow)

{'ElementFilter': ElementFilter(component_name='ElementFilter', component_description='Filter out molecules who contain elements not in the allowed element list', component_fail_message='Molecule contained elements not in the allowed elements list', allowed_elements=[1, 6, 7, 8]), 'MolecularWeightFilter': MolecularWeightFilter(component_name='MolecularWeightFilter', component_description='Molecules are filtered based on the allowed molecular weights.', component_fail_message='Molecule weight was not in the specified region.', minimum_weight=130, maximum_weight=781), 'StandardConformerGenerator': StandardConformerGenerator(component_name='StandardConformerGenerator', component_description='Generate conformations for the given molecules', component_fail_message='Conformers could not be generated', max_conformers=20, clear_exsiting=True, toolkit='rdkit')}


In [10]:
# now lets save the workflow to json
factory.export_settings('work1.json')

# and lets save to yaml
factory.export_settings('work1.yaml')

In [11]:
# lets look at the ouput files 
! head -n 20 work1.yaml

basis: DZVP
client: public
driver: energy
maxiter: 200
priority: normal
program: psi4
scf_properties:
- dipole
- qudrupole
- wiberg_lowdin_indices
tag: openff
theory: wB97x-d
workflow:
  ElementFilter:
    allowed_elements:
    - 1
    - 6
    - 7
    - 8
    component_description: Filter out molecules who contain elements not in the allowed


In [12]:
! head -n 20 work1.json

{
  "theory": "wB97x-d",
  "basis": "DZVP",
  "program": "psi4",
  "maxiter": 200,
  "driver": "energy",
  "scf_properties": [
    "dipole",
    "qudrupole",
    "wiberg_lowdin_indices"
  ],
  "client": "public",
  "priority": "normal",
  "tag": "openff",
  "workflow": {
    "ElementFilter": {
      "component_name": "ElementFilter",
      "component_description": "Filter out molecules who contain elements not in the allowed element list",
      "component_fail_message": "Molecule contained elements not in the allowed elements list",
      "allowed_elements": [


Now lets make a new workflow factory and load in the settings we just saved to quickly make a new workflow.


In [13]:
factory2 = QCFractalDatasetFactory()

# now lets print out the basic workflow factory
print(factory2)

theory='B3LYP-D3BJ' basis='DZVP' program='psi4' maxiter=200 driver='energy' scf_properties=['dipole', 'qudrupole', 'wiberg_lowdin_indices'] client='public' priority='normal' tag='openff' workflow={}


In [14]:
# now load in the settings from the yaml file and print out the factory components

factory2.import_settings('work1.yaml')

# note that the theory has changed along with making the workflow components
print(factory2)

theory='wB97x-d' basis='DZVP' program='psi4' maxiter=200 driver='energy' scf_properties=['dipole', 'qudrupole', 'wiberg_lowdin_indices'] client='public' priority='normal' tag='openff' workflow={'ElementFilter': ElementFilter(component_name='ElementFilter', component_description='Filter out molecules who contain elements not in the allowed element list', component_fail_message='Molecule contained elements not in the allowed elements list', allowed_elements=[1, 6, 7, 8]), 'MolecularWeightFilter': MolecularWeightFilter(component_name='MolecularWeightFilter', component_description='Molecules are filtered based on the allowed molecular weights.', component_fail_message='Molecule weight was not in the specified region.', minimum_weight=130, maximum_weight=781), 'StandardConformerGenerator': StandardConformerGenerator(component_name='StandardConformerGenerator', component_description='Generate conformations for the given molecules', component_fail_message='Conformers could not be generated', 

Now we can run the worklow on a set of molecules in the mini drug bank file that comes with the openforcefield.

In [15]:
mols = Molecule.from_file(get_data_file_path('molecules/minidrugbank.sdf'), allow_undefined_stereo=True)

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): DrugBank_2987
Problematic atoms are:
Atom atomic num: 7, name: , idx: 20, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 4, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 16, aromatic: False, chiral: True
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 28, aromatic: False, chiral: False

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): DrugBank_3502
Problematic atoms are:
Atom atomic num: 7, name: , idx: 20, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 4, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 16, aromatic: False, chiral: True
bond order: 1, chiral: False to atom atomic n

In [16]:
# create the dataset ready for submission
dataset = factory2.create_dataset('my_dataset', mols)

Warning (not error because allow_undefined_stereo=True): 
RDKit ERROR: [16:36:15] Explicit valence for atom # 35 O, 3, is greater than permitted
RDKit ERROR: [16:36:34] Explicit valence for atom # 11 N, 5, is greater than permitted
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): DrugBank_6531
Problematic atoms are:
Atom atomic num: 7, name: , idx: 23, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 2, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 14, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 18, aromatic: False, chiral: False



In [17]:
dataset.dataset

{'c1ccc(cc1)C[C@H](CC(CN(=O)O)(O)O)C(=O)O': {'attributes': {'canonical_smiles': 'c1ccc(cc1)CC(CC(CN(=O)O)(O)O)C(=O)O',
   'canonical_isomeric_smiles': 'c1ccc(cc1)C[C@H](CC(CN(=O)O)(O)O)C(=O)O',
   'canonical_explicit_hydrogen_smiles': '[H]c1c(c(c(c(c1[H])[H])C([H])([H])C([H])(C(=O)O[H])C([H])([H])C(C([H])([H])N(=O)([H])O[H])(O[H])O[H])[H])[H]',
   'canonical_isomeric_explicit_hydrogen_smiles': '[H]c1c(c(c(c(c1[H])[H])C([H])([H])[C@@]([H])(C(=O)O[H])C([H])([H])C(C([H])([H])N(=O)([H])O[H])(O[H])O[H])[H])[H]',
   'canonical_isomeric_explicit_hydrogen_mapped_smiles': '[H:20][c:1]1[c:2]([c:4]([c:6]([c:5]([c:3]1[H:22])[H:24])[C:8]([H:25])([H:26])[C@@:11]([H:31])([C:7](=[O:14])[O:16][H:33])[C:9]([H:27])([H:28])[C:12]([C:10]([H:29])([H:30])[N:13](=[O:15])([H:32])[O:19][H:36])([O:17][H:34])[O:18][H:35])[H:23])[H:21]',
   'molecular_formula': 'C12H17NO6',
   'standard_inchi': 'InChI=1S/C12H17NO6/c14-11(15)10(6-9-4-2-1-3-5-9)7-12(16,17)8-13(18)19/h1-5,10,13,16-17H,6-8H2,(H,14,15)(H,18,19)/t10-/m1

In [18]:
for molecule in dataset.filtered:
    print(molecule)

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 7, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 6, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 8, aromatic: False, chiral: True
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 48, aromatic: False, chiral: False



Molecule with name '' and SMILES '[H]c1c(c(c(c(c1[H])[H])Oc2c(c(c(c(c2[H])C([H])([H])C([H])([H])C([H])([H])[C@]([H])(P(=O)(O[H])O[H])S(=O)(=O)O[H])[H])[H])[H])[H])[H]'
Molecule with name '' and SMILES '[H]c1c(c(c(c(c1C([H])([H])[H])[H])[H])S(=O)(=O)O[H])[H]'
Molecule with name '' and SMILES '[H]c1c(c(c(c(c1C(C([H])([H])[H])(C([H])([H])[H])C([H])([H])[H])[H])[H])S(=O)(=O)O[H])[H]'
Molecule with name '' and SMILES '[H]c1c(c(c(c(c1C([H])([H])O[C@@]([H])(O[H])SC([H])([H])[C@]([H])(C(=O)N([H])C([H])([H])C(=O)O[H])N([H])C(=O)C([H])([H])C([H])([H])[C@@]([H])(C(=O)O[H])N([H])[H])[H])[H])N(=O)([H])O[H])[H]'
Molecule with name '' and SMILES '[H]c1c(c(c2c(c1[H])N=C(S2)SC([H])([H])C([H])([H])C([H])([H])S(=O)(=O)O[H])[H])[H]'
Molecule with name '' and SMILES '[H]c1c(c(c(c(c1c2c(c(c(c(c2[H])[H])[C@@]([H])(C([H])([H])[H])C([H])([H])N([H])S(=O)(=O)C([H])([H])[H])[H])[H])[H])[H])[C@@]([H])(C([H])([H])[H])C([H])([H])N([H])S(=O)(=O)C([H])([H])[H])[H]'
Molecule with name '' and SMILES '[H]c1c(c2c(c(c(c(c2

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 7, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 6, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 8, aromatic: False, chiral: True
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 48, aromatic: False, chiral: False



Molecule with name '' and SMILES '[H]C1=C(N(C(C(=C1N([H])C([H])([H])C([H])([H])[C@]([H])(C(=O)N2C(C(C(C(C2([H])[H])([H])[H])([H])[H])([H])[H])([H])[H])N([H])C(=O)C([H])([H])N([H])S(=O)(=O)C3(C(C(C(C(C3([H])[H])([H])[H])([H])[H])([H])[H])([H])[H])[H])[H])([H])[H])[H])[H]'
Molecule with name '' and SMILES '[H]/C(=C(/[H])\S[H])/N([H])C(=O)C([H])([H])C([H])([H])N([H])C(=O)[C@@]([H])(C(C([H])([H])[H])(C([H])([H])[H])C([H])([H])O[H])O[H]'
Molecule with name '' and SMILES '[H]c1c(c(c(c(c1C([H])([H])[C@@]([H])(C(=O)O[H])N([H])S(=O)(=O)C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])[H])[H])OC([H])([H])C([H])([H])C([H])([H])C([H])([H])C2(C(C(N(C(C2([H])[H])([H])[H])[H])([H])[H])([H])[H])[H])[H]'
Molecule with name '' and SMILES '[H]c1c(c(c(c(c1C([H])([H])C(=O)OC([H])([H])C([H])([H])Br)C(=O)[H])[H])N([H])[H])[H]'
Molecule with name '' and SMILES '[H][C@](C(=O)O[H])(C([H])([H])C([H])([H])S(=O)(=O)C([H])([H])[H])N([H])[H]'
Molecule with name '' and SMILES '[H]C([H])([H])C([H])([H])C([H])([H])C([H]

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 11, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 3, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 12, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 7, name: , idx: 32, aromatic: False, chiral: False

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 11, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 3, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 12, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 7, name: , idx: 32, 

Molecule with name '' and SMILES '[H]c1c(c(c(c(c1[H])[H])C([H])([H])[N@@]([C@@]([H])(C(=O)N([H])O[H])C([H])([H])C([H])(C([H])([H])[H])C([H])([H])[H])[P@](=O)(c2c(c(c(c(c2[H])[H])[H])[H])[H])C([H])([H])[H])[H])[H]'
Molecule with name '' and SMILES '[H]c1c(c(c(c(c1[H])[H])N([H])c2c(c(c(c3c2c(c(c(c3[H])[H])[H])S(=O)(=O)O[H])[H])[H])[H])[H])[H]'
Molecule with name '' and SMILES '[H]C([H])([H])C([H])(C([H])([H])[H])OP(=O)([H])OC([H])(C([H])([H])[H])C([H])([H])[H]'
Molecule with name '' and SMILES '[H]c1c(c(c(c(c1[H])[H])C([H])([H])[C@@]([H])(C(=O)N([H])[C@]([H])(C([H])([H])C([H])([H])c2c(c(c(c(c2[H])[H])[H])[H])[H])C([H])([H])C([H])([H])S(=O)(=O)Oc3c(c(c(c(c3[H])[H])[H])[H])[H])N([H])C(=O)OC([H])([H])c4c(c(c(c(c4[H])[H])[H])[H])[H])[H])[H]'
Molecule with name '' and SMILES '[H]c1c(c(c(c(c1C(=O)N([H])c2c(c(nc(c2Cl)[H])[H])Cl)[H])OC([H])([H])C3(C(C3([H])[H])([H])[H])[H])OC([H])(F)F)[H]'
Molecule with name '' and SMILES '[H]C1(C(C(C(C(C1([H])OS(=O)(=O)O[H])([H])OS(=O)(=O)O[H])([H])OS(=O)(=O)O[

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 3, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 2, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 4, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 61, aromatic: False, chiral: False



Molecule with name '' and SMILES '[H]c1c(c(c(c(n1)C([H])([H])[H])O[H])C([H])([H])N([H])[C@]2(C(=O)N(OC2([H])[H])[H])[H])C([H])([H])OP(=O)(O[H])O[H]'
Molecule with name '' and SMILES '[H]c1c(c(c2c(c1[H])c(nnc2N([H])c3c(c(c(c(c3[H])[H])Cl)[H])[H])C([H])([H])c4c(c(nc(c4[H])[H])[H])[H])[H])[H]'
Molecule with name '' and SMILES '[H]c1c(c(c(c(c1[H])[H])C([H])([H])[C@]([H])(C(=O)N([H])[C@]([H])(C([H])([H])C([H])([H])c2c(c(c(c(c2[H])[H])[H])[H])[H])C([H])([H])C([H])([H])S(=O)(=O)Oc3c(c(c(c(c3[H])[H])N(=O)([H])O[H])[H])[H])N([H])C(=O)OC([H])([H])c4c(c(c(c(c4[H])[H])[H])[H])[H])[H])[H]'
Molecule with name '' and SMILES '[H]c1c(c(c2c(c1[H])C(=C([N@@](S2(=O)=O)C([H])([H])[H])C(=O)/N=C/3\C(=C(ON3[H])C([H])([H])[H])[H])O[H])[H])[H]'
Molecule with name '' and SMILES '[H]C1=C([C@]2(C(=C(C1=O)[H])[C@@](C([C@@]3([C@@]2([C@@](C([C@]4([C@]3(C([C@@]5([C@]4(OC(O5)(C([H])([H])[H])C([H])([H])[H])C(=O)C([H])([H])O[H])[H])([H])[H])[H])C([H])([H])[H])([H])[H])([H])Cl)Cl)[H])([H])[H])([H])F)C([H])([H])[H])[H]'
Mo

Molecule with name '' and SMILES '[H]c1c(c(c(c2c1C([C@@](N(C2([H])[H])C(=O)OC(C([H])([H])[H])(C([H])([H])[H])C([H])([H])[H])([H])C(=O)N([H])C([H])([H])[H])([H])[H])[H])N([H])S(=O)(=O)O[H])[H]'
Molecule with name '' and SMILES '[H]C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])S(=O)(=O)O[H]'
Molecule with name '' and SMILES '[H]C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])[P@](=O)([H])OC([H])([H])C([H])([H])C([H])([H])C([H])([H])[H]'
Molecule with name '' and SMILES '[H]/C(=C(/[H])\C([H])([H])/C(=C(\[H])/C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])/[H])/C([H])([H])/C(=C(\[H])/C([H])([H])/C(=C(\[H])/C([H])([H])C([H])([H])C([H])([H])C([H])([H])[P@@](=O)(OC([H])([H])[H])F)/[H])/[H]'
Molecule with name '' and SMILES '[H][C@@]1([C@](C([N@]([C@]1([H])C([H])([H])O[H])C([H])([H])[C@@]([H])([C@]([H])(C([H])([H])O[H])OS(O[H])(O[H])O[H])O[H])([H]

Molecule with name '' and SMILES '[H]C1=C(ON=N1([H])N2C(C(OC(C2([H])[H])([H])[H])([H])[H])([H])[H])N([H])C(=O)OC([H])([H])C([H])([H])[H]'


In [19]:
for molecule in dataset.molecules:
    print(molecule)

Molecule with name 'c1ccc(cc1)C[C@H](CC(CN(=O)O)(O)O)C(=O)O' and SMILES '[H]c1c(c(c(c(c1[H])[H])C([H])([H])[C@@]([H])(C(=O)O[H])C([H])([H])C(C([H])([H])N(=O)([H])O[H])(O[H])O[H])[H])[H]'
Molecule with name 'c1ccc(cc1)C[C@H](CC(CN(=O)O)(O)O)C(=O)O_1' and SMILES '[H]c1c(c(c(c(c1[H])[H])C([H])([H])[C@@]([H])(C(=O)O[H])C([H])([H])C(C([H])([H])N(=O)([H])O[H])(O[H])O[H])[H])[H]'
Molecule with name 'c1ccc(cc1)C[C@H](CC(CN(=O)O)(O)O)C(=O)O_1_2' and SMILES '[H]c1c(c(c(c(c1[H])[H])C([H])([H])[C@@]([H])(C(=O)O[H])C([H])([H])C(C([H])([H])N(=O)([H])O[H])(O[H])O[H])[H])[H]'
Molecule with name 'c1ccc(cc1)C[C@H](CC(CN(=O)O)(O)O)C(=O)O_1_2_3' and SMILES '[H]c1c(c(c(c(c1[H])[H])C([H])([H])[C@@]([H])(C(=O)O[H])C([H])([H])C(C([H])([H])N(=O)([H])O[H])(O[H])O[H])[H])[H]'
Molecule with name 'c1ccc(cc1)C[C@H](CC(CN(=O)O)(O)O)C(=O)O_1_2_3_4' and SMILES '[H]c1c(c(c(c(c1[H])[H])C([H])([H])[C@@]([H])(C(=O)O[H])C([H])([H])C(C([H])([H])N(=O)([H])O[H])(O[H])O[H])[H])[H]'
Molecule with name 'c1ccc(cc1)C[C@H](CC(CN(=O

Molecule with name 'c1ccc(cc1)C2=NN3C=CC=CC3=C2c4cc5c(nn4)NN=C5N_1_2' and SMILES '[H]c1c(c(c(c(c1[H])[H])C2=NN3C(=C(C(=C(C3=C2c4c(c5c(nn4)N(N=C5N([H])[H])[H])[H])[H])[H])[H])[H])[H])[H]'
Molecule with name 'c1ccc(cc1)C2=NN3C=CC=CC3=C2c4cc5c(nn4)NN=C5N_1_2_3' and SMILES '[H]c1c(c(c(c(c1[H])[H])C2=NN3C(=C(C(=C(C3=C2c4c(c5c(nn4)N(N=C5N([H])[H])[H])[H])[H])[H])[H])[H])[H])[H]'
Molecule with name 'c1ccc(cc1)C2=NN3C=CC=CC3=C2c4cc5c(nn4)NN=C5N_1_2_3_4' and SMILES '[H]c1c(c(c(c(c1[H])[H])C2=NN3C(=C(C(=C(C3=C2c4c(c5c(nn4)N(N=C5N([H])[H])[H])[H])[H])[H])[H])[H])[H])[H]'
Molecule with name 'c1ccc(cc1)C2=NN3C=CC=CC3=C2c4cc5c(nn4)NN=C5N_1_2_3_4_5' and SMILES '[H]c1c(c(c(c(c1[H])[H])C2=NN3C(=C(C(=C(C3=C2c4c(c5c(nn4)N(N=C5N([H])[H])[H])[H])[H])[H])[H])[H])[H])[H]'
Molecule with name 'C(CCCN)CCCNC(N)N' and SMILES '[H]C([H])(C([H])([H])C([H])([H])C([H])([H])N([H])[H])C([H])([H])C([H])([H])C([H])([H])N([H])C([H])(N([H])[H])N([H])[H]'
Molecule with name 'C(CCCN)CCCNC(N)N_1' and SMILES '[H]C([H])(C([H])([

Molecule with name 'CCCC[C@H](CNNC[C@H](CCCC)C(=O)N[C@H](CCCCN)C(=O)Nc1ccccc1)C(=O)N[C@@H](CCCCN)C(=O)Nc2ccccc2_1_2_3_4_5_6_7' and SMILES '[H]c1c(c(c(c(c1[H])[H])N([H])C(=O)[C@]([H])(C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])[H])N([H])C(=O)[C@]([H])(C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])C([H])([H])N([H])N([H])C([H])([H])[C@@]([H])(C(=O)N([H])[C@@]([H])(C(=O)N([H])c2c(c(c(c(c2[H])[H])[H])[H])[H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])[H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])[H])[H]'
Molecule with name 'CCCC[C@H](CNNC[C@H](CCCC)C(=O)N[C@H](CCCCN)C(=O)Nc1ccccc1)C(=O)N[C@@H](CCCCN)C(=O)Nc2ccccc2_1_2_3_4_5_6_7_8' and SMILES '[H]c1c(c(c(c(c1[H])[H])N([H])C(=O)[C@]([H])(C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])[H])N([H])C(=O)[C@]([H])(C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])C([H])([H])N([H])N([H])C([H])([H])[C@@]([H])(C(=O)N([H])[C@@]([H])(C(=O)N([H])c2c(c(c(c(c2[H])[H])[H])[H])[H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])[H])C

Molecule with name 'Cc1cc(nnc1NCCN2CCOCC2)c3ccccc3_1_2_3_4_5_6_7_8_9' and SMILES '[H]c1c(c(c(c(c1[H])[H])c2c(c(c(nn2)N([H])C([H])([H])C([H])([H])N3C(C(OC(C3([H])[H])([H])[H])([H])[H])([H])[H])C([H])([H])[H])[H])[H])[H]'
Molecule with name 'Cc1cc(nnc1NCCN2CCOCC2)c3ccccc3_1_2_3_4_5_6_7_8_9_10' and SMILES '[H]c1c(c(c(c(c1[H])[H])c2c(c(c(nn2)N([H])C([H])([H])C([H])([H])N3C(C(OC(C3([H])[H])([H])[H])([H])[H])([H])[H])C([H])([H])[H])[H])[H])[H]'
Molecule with name 'Cc1cc(nnc1NCCN2CCOCC2)c3ccccc3_1_2_3_4_5_6_7_8_9_10_11' and SMILES '[H]c1c(c(c(c(c1[H])[H])c2c(c(c(nn2)N([H])C([H])([H])C([H])([H])N3C(C(OC(C3([H])[H])([H])[H])([H])[H])([H])[H])C([H])([H])[H])[H])[H])[H]'
Molecule with name 'Cc1cc(nnc1NCCN2CCOCC2)c3ccccc3_1_2_3_4_5_6_7_8_9_10_11_12' and SMILES '[H]c1c(c(c(c(c1[H])[H])c2c(c(c(nn2)N([H])C([H])([H])C([H])([H])N3C(C(OC(C3([H])[H])([H])[H])([H])[H])([H])[H])C([H])([H])[H])[H])[H])[H]'
Molecule with name 'Cc1cc(nnc1NCCN2CCOCC2)c3ccccc3_1_2_3_4_5_6_7_8_9_10_11_12_13' and SMILES '[H]c1c(c

Molecule with name 'CCOC(=O)c1ccc(cc1)OCCCC2CCN(CC2)c3ccc(nn3)C_1_2_3' and SMILES '[H]c1c(c(c(c(c1C(=O)OC([H])([H])C([H])([H])[H])[H])[H])OC([H])([H])C([H])([H])C([H])([H])C2(C(C(N(C(C2([H])[H])([H])[H])c3c(c(c(nn3)C([H])([H])[H])[H])[H])([H])[H])([H])[H])[H])[H]'
Molecule with name 'CCOC(=O)c1ccc(cc1)OCCCC2CCN(CC2)c3ccc(nn3)C_1_2_3_4' and SMILES '[H]c1c(c(c(c(c1C(=O)OC([H])([H])C([H])([H])[H])[H])[H])OC([H])([H])C([H])([H])C([H])([H])C2(C(C(N(C(C2([H])[H])([H])[H])c3c(c(c(nn3)C([H])([H])[H])[H])[H])([H])[H])([H])[H])[H])[H]'
Molecule with name 'CCOC(=O)c1ccc(cc1)OCCCC2CCN(CC2)c3ccc(nn3)C_1_2_3_4_5' and SMILES '[H]c1c(c(c(c(c1C(=O)OC([H])([H])C([H])([H])[H])[H])[H])OC([H])([H])C([H])([H])C([H])([H])C2(C(C(N(C(C2([H])[H])([H])[H])c3c(c(c(nn3)C([H])([H])[H])[H])[H])([H])[H])([H])[H])[H])[H]'
Molecule with name 'CCOC(=O)c1ccc(cc1)OCCCC2CCN(CC2)c3ccc(nn3)C_1_2_3_4_5_6' and SMILES '[H]c1c(c(c(c(c1C(=O)OC([H])([H])C([H])([H])[H])[H])[H])OC([H])([H])C([H])([H])C([H])([H])C2(C(C(N(C(C2([H])[H]

...

Molecule with name 'C1CC[C@H]2[C@H](C1)CC[N@@]2C[C@H]3CC[C@H]4[C@@H](C3)[C@@H](N[C@@H](N4)N)N_1_2_3_4_5_6_7_8_9_10_11_12_13' and SMILES '[H][C@]12[C@](C(C([C@](C1([H])[H])([H])C([H])([H])[N@@]3[C@@]4([C@](C(C(C(C4([H])[H])([H])[H])([H])[H])([H])[H])(C(C3([H])[H])([H])[H])[H])[H])([H])[H])([H])[H])(N([C@@](N([C@@]2([H])N([H])[H])[H])([H])N([H])[H])[H])[H]'
Molecule with name 'C1CC[C@H]2[C@H](C1)CC[N@@]2C[C@H]3CC[C@H]4[C@@H](C3)[C@@H](N[C@@H](N4)N)N_1_2_3_4_5_6_7_8_9_10_11_12_13_14' and SMILES '[H][C@]12[C@](C(C([C@](C1([H])[H])([H])C([H])([H])[N@@]3[C@@]4([C@](C(C(C(C4([H])[H])([H])[H])([H])[H])([H])[H])(C(C3([H])[H])([H])[H])[H])[H])([H])[H])([H])[H])(N([C@@](N([C@@]2([H])N([H])[H])[H])([H])N([H])[H])[H])[H]'
Molecule with name 'C1CC[C@H]2[C@H](C1)CC[N@@]2C[C@H]3CC[C@H]4[C@@H](C3)[C@@H](N[C@@H](N4)N)N_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15' and SMILES '[H][C@]12[C@](C(C([C@](C1([H])[H])([H])C([H])([H])[N@@]3[C@@]4([C@](C(C(C(C4([H])[H])([H])[H])([H])[H])([H])[H])(C(C3([H])[H])([H])[H])[H]

Molecule with name 'c1ccc(cc1)C(=C/C=N/c2c3ccccc3nnc2CN)c4ccccc4_1_2' and SMILES '[H]c1c(c(c(c(c1[H])[H])C(=C([H])/C(=N/c2c3c(c(c(c(c3nnc2C([H])([H])N([H])[H])[H])[H])[H])[H])/[H])c4c(c(c(c(c4[H])[H])[H])[H])[H])[H])[H]'
Molecule with name 'c1ccc(cc1)C(=C/C=N/c2c3ccccc3nnc2CN)c4ccccc4_1_2_3' and SMILES '[H]c1c(c(c(c(c1[H])[H])C(=C([H])/C(=N/c2c3c(c(c(c(c3nnc2C([H])([H])N([H])[H])[H])[H])[H])[H])/[H])c4c(c(c(c(c4[H])[H])[H])[H])[H])[H])[H]'
Molecule with name 'c1ccc(cc1)C(=C/C=N/c2c3ccccc3nnc2CN)c4ccccc4_1_2_3_4' and SMILES '[H]c1c(c(c(c(c1[H])[H])C(=C([H])/C(=N/c2c3c(c(c(c(c3nnc2C([H])([H])N([H])[H])[H])[H])[H])[H])/[H])c4c(c(c(c(c4[H])[H])[H])[H])[H])[H])[H]'
Molecule with name 'c1ccc(cc1)C(=C/C=N/c2c3ccccc3nnc2CN)c4ccccc4_1_2_3_4_5' and SMILES '[H]c1c(c(c(c(c1[H])[H])C(=C([H])/C(=N/c2c3c(c(c(c(c3nnc2C([H])([H])N([H])[H])[H])[H])[H])[H])/[H])c4c(c(c(c(c4[H])[H])[H])[H])[H])[H])[H]'
Molecule with name 'c1ccc(cc1)C(=C/C=N/c2c3ccccc3nnc2CN)c4ccccc4_1_2_3_4_5_6' and SMILES '[H]c1c(c(c(c(c

Molecule with name 'CCCCCN[C@@H]([C@@H](Cc1ccc(c(c1)C(=O)O)OCC(=O)O)N[C@H]([C@@H](Cc2ccccc2)N[C@H](O)OC(C)(C)C)O)O_1_2' and SMILES '[H]c1c(c(c(c(c1[H])[H])C([H])([H])[C@]([H])([C@@]([H])(N([H])[C@@]([H])([C@]([H])(N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])O[H])C([H])([H])c2c(c(c(c(c2[H])C(=O)O[H])OC([H])([H])C(=O)O[H])[H])[H])O[H])N([H])[C@]([H])(O[H])OC(C([H])([H])[H])(C([H])([H])[H])C([H])([H])[H])[H])[H]'
Molecule with name 'CCCCCN[C@@H]([C@@H](Cc1ccc(c(c1)C(=O)O)OCC(=O)O)N[C@H]([C@@H](Cc2ccccc2)N[C@H](O)OC(C)(C)C)O)O_1_2_3' and SMILES '[H]c1c(c(c(c(c1[H])[H])C([H])([H])[C@]([H])([C@@]([H])(N([H])[C@@]([H])([C@]([H])(N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])O[H])C([H])([H])c2c(c(c(c(c2[H])C(=O)O[H])OC([H])([H])C(=O)O[H])[H])[H])O[H])N([H])[C@]([H])(O[H])OC(C([H])([H])[H])(C([H])([H])[H])C([H])([H])[H])[H])[H]'
Molecule with name 'CCCCCN[C@@H]([C@@H](Cc1ccc(c(c1)C(=O)O)OCC(=O)O)N[C@H]([C@@H](Cc2ccccc2)N[C@H](O)OC(C)(C)C)O)O_1_2_3_4' and 

Molecule with name 'C/C(=C\C=C\C=C\C=C(/C)\C=C/1\C=C(C(=O)O1)/C=C/[C@@]23[C@@](O2)(C[C@@H](CC3(C)C)O)C)/C=C=C4[C@@](C[C@@H](CC4(C)C)OC(=O)C)(C)O' and SMILES '[H]C/1=C(C(=O)O\C1=C(\[H])/C(=C(\[H])/C(=C(\[H])/C(=C(\[H])/C(=C(/C(=C=C2[C@@](C([C@](C(C2(C([H])([H])[H])C([H])([H])[H])([H])[H])([H])OC(=O)C([H])([H])[H])([H])[H])(C([H])([H])[H])O[H])[H])\C([H])([H])[H])/[H])/[H])/[H])/C([H])([H])[H])/C(=C(\[H])/[C@@]34[C@@](O3)(C([C@](C(C4(C([H])([H])[H])C([H])([H])[H])([H])[H])([H])O[H])([H])[H])C([H])([H])[H])/[H]'
Molecule with name 'C/C(=C\C=C\C=C\C=C(/C)\C=C/1\C=C(C(=O)O1)/C=C/[C@@]23[C@@](O2)(C[C@@H](CC3(C)C)O)C)/C=C=C4[C@@](C[C@@H](CC4(C)C)OC(=O)C)(C)O_1' and SMILES '[H]C/1=C(C(=O)O\C1=C(\[H])/C(=C(\[H])/C(=C(\[H])/C(=C(\[H])/C(=C(/C(=C=C2[C@@](C([C@](C(C2(C([H])([H])[H])C([H])([H])[H])([H])[H])([H])OC(=O)C([H])([H])[H])([H])[H])(C([H])([H])[H])O[H])[H])\C([H])([H])[H])/[H])/[H])/[H])/C([H])([H])[H])/C(=C(\[H])/[C@@]34[C@@](O3)(C([C@](C(C4(C([H])([H])[H])C([H])([H])[H])([H])[H])([H])O[H

Molecule with name 'C/C(=C\C=C\C=C\C=C(/C)\C=C/1\C=C(C(=O)O1)/C=C/[C@@]23[C@@](O2)(C[C@@H](CC3(C)C)O)C)/C=C=C4[C@@](C[C@@H](CC4(C)C)OC(=O)C)(C)O_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18' and SMILES '[H]C/1=C(C(=O)O\C1=C(\[H])/C(=C(\[H])/C(=C(\[H])/C(=C(\[H])/C(=C(/C(=C=C2[C@@](C([C@](C(C2(C([H])([H])[H])C([H])([H])[H])([H])[H])([H])OC(=O)C([H])([H])[H])([H])[H])(C([H])([H])[H])O[H])[H])\C([H])([H])[H])/[H])/[H])/[H])/C([H])([H])[H])/C(=C(\[H])/[C@@]34[C@@](O3)(C([C@](C(C4(C([H])([H])[H])C([H])([H])[H])([H])[H])([H])O[H])([H])[H])C([H])([H])[H])/[H]'
Molecule with name 'C/C(=C\C=C\C=C\C=C(/C)\C=C/1\C=C(C(=O)O1)/C=C/[C@@]23[C@@](O2)(C[C@@H](CC3(C)C)O)C)/C=C=C4[C@@](C[C@@H](CC4(C)C)OC(=O)C)(C)O_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18_19' and SMILES '[H]C/1=C(C(=O)O\C1=C(\[H])/C(=C(\[H])/C(=C(\[H])/C(=C(\[H])/C(=C(/C(=C=C2[C@@](C([C@](C(C2(C([H])([H])[H])C([H])([H])[H])([H])[H])([H])OC(=O)C([H])([H])[H])([H])[H])(C([H])([H])[H])O[H])[H])\C([H])([H])[H])/[H])/[H])/[H])/C([H])([H])[

In [22]:
dataset.components

[{'component_name': 'ElementFilter',
  'component_description': 'Filter out molecules who contain elements not in the allowed element list',
  'component_fail_message': 'Molecule contained elements not in the allowed elements list',
  'allowed_elements': [1, 6, 7, 8]},
 {'component_name': 'MolecularWeightFilter',
  'component_description': 'Molecules are filtered based on the allowed molecular weights.',
  'component_fail_message': 'Molecule weight was not in the specified region.',
  'minimum_weight': 130,
  'maximum_weight': 781},
 {'component_name': 'StandardConformerGenerator',
  'component_description': 'Generate conformations for the given molecules',
  'component_fail_message': 'Conformers could not be generated',
  'max_conformers': 20,
  'clear_exsiting': True,
  'toolkit': 'rdkit'}]

In [23]:
dataset.n_molecules

840

In [24]:
dataset.n_records

840

In [25]:
len(mols)

371

In [26]:
len(dataset.dataset.keys())

840

In [27]:
dataset.filtered_molecules

{'ElementFilter': {'component_description': {'component_name': 'ElementFilter',
   'component_description': 'Filter out molecules who contain elements not in the allowed element list',
   'component_fail_message': 'Molecule contained elements not in the allowed elements list',
   'allowed_elements': [1, 6, 7, 8]},
  'molecules': ['[H]c1c(c(c(c(c1[H])[H])Oc2c(c(c(c(c2[H])C([H])([H])C([H])([H])C([H])([H])[C@]([H])(P(=O)(O[H])O[H])S(=O)(=O)O[H])[H])[H])[H])[H])[H]',
   '[H]c1c(c(c(c(c1C([H])([H])[H])[H])[H])S(=O)(=O)O[H])[H]',
   '[H]c1c(c(c(c(c1C(C([H])([H])[H])(C([H])([H])[H])C([H])([H])[H])[H])[H])S(=O)(=O)O[H])[H]',
   '[H]c1c(c(c(c(c1C([H])([H])O[C@@]([H])(O[H])SC([H])([H])[C@]([H])(C(=O)N([H])C([H])([H])C(=O)O[H])N([H])C(=O)C([H])([H])C([H])([H])[C@@]([H])(C(=O)O[H])N([H])[H])[H])[H])N(=O)([H])O[H])[H]',
   '[H]c1c(c(c2c(c1[H])N=C(S2)SC([H])([H])C([H])([H])C([H])([H])S(=O)(=O)O[H])[H])[H]',
   '[H]c1c(c(c(c(c1c2c(c(c(c(c2[H])[H])[C@@]([H])(C([H])([H])[H])C([H])([H])N([H])S(=O)(=O)C(

In [30]:
for i in dataset.dataset['c1ccc(cc1)C[C@H](CC(CN(=O)O)(O)O)C(=O)O']['initial_molecules']:
    print(i)

('schema_name', 'qcschema_molecule')
('schema_version', 2)
('validated', True)
('symbols', array(['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'N',
       'O', 'O', 'O', 'O', 'O', 'O', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'], dtype='<U1'))
('geometry', array([[-7.49487608e+00,  3.01688115e+00,  2.93843166e+00],
       [-7.19254227e+00,  6.18293370e-01,  3.80461569e+00],
       [-6.40734138e+00,  3.70846110e+00,  6.35657070e-01],
       [-5.81231093e+00, -1.10934123e+00,  2.39687244e+00],
       [-5.04028219e+00,  1.94846784e+00, -7.30255310e-01],
       [-4.71573872e+00, -4.92930580e-01,  1.22014000e-01],
       [ 6.87470350e-01, -4.43548120e+00, -2.21220677e+00],
       [-3.27302231e+00, -2.38807080e+00, -1.29748973e+00],
       [ 7.50425060e-01,  8.63509000e-02, -1.02418278e+00],
       [ 4.68648523e+00,  2.52198315e+00, -7.07789800e-01],
       [-5.22740150e-01, -2.44857257e+00, -6.46021320e-01],
       [ 3.51161786e

In [28]:
dataset.filtered_molecules

{'ElementFilter': {'component_description': {'component_name': 'ElementFilter',
   'component_description': 'Filter out molecules who contain elements not in the allowed element list',
   'component_fail_message': 'Molecule contained elements not in the allowed elements list',
   'allowed_elements': [1, 6, 7, 8]},
  'molecules': ['[H]c1c(c(c(c(c1[H])[H])Oc2c(c(c(c(c2[H])C([H])([H])C([H])([H])C([H])([H])[C@]([H])(P(=O)(O[H])O[H])S(=O)(=O)O[H])[H])[H])[H])[H])[H]',
   '[H]c1c(c(c(c(c1C([H])([H])[H])[H])[H])S(=O)(=O)O[H])[H]',
   '[H]c1c(c(c(c(c1C(C([H])([H])[H])(C([H])([H])[H])C([H])([H])[H])[H])[H])S(=O)(=O)O[H])[H]',
   '[H]c1c(c(c(c(c1C([H])([H])O[C@@]([H])(O[H])SC([H])([H])[C@]([H])(C(=O)N([H])C([H])([H])C(=O)O[H])N([H])C(=O)C([H])([H])C([H])([H])[C@@]([H])(C(=O)O[H])N([H])[H])[H])[H])N(=O)([H])O[H])[H]',
   '[H]c1c(c(c2c(c1[H])N=C(S2)SC([H])([H])C([H])([H])C([H])([H])S(=O)(=O)O[H])[H])[H]',
   '[H]c1c(c(c(c(c1c2c(c(c(c(c2[H])[H])[C@@]([H])(C([H])([H])[H])C([H])([H])N([H])S(=O)(=O)C(